In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2,3"
import numpy as np
import pandas as pd
from keras.models import *
from keras.layers import Input, merge, Conv2D, MaxPooling2D, UpSampling2D, Dropout, Cropping2D, concatenate, Activation,Conv2DTranspose
from keras.layers import BatchNormalization
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, History, EarlyStopping, ReduceLROnPlateau
from keras import backend as keras
from keras.utils.multi_gpu_utils import multi_gpu_model
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import numpy as np
import os
import glob
import shutil
import cv2
from segmentation_GP import *

Using TensorFlow backend.


In [2]:
def mkfolder(folder):

    for j in range(len(folder)):
        if not os.path.lexists(folder[j]):
            os.makedirs(folder[j])

In [3]:
## train data ##
data_path = './3_deepdata/1_exp1/train/LAT_aug_img/'
label_path = './3_deepdata/1_exp1/train/LAT_aug_label/'

## test data ##
test_path = './3_deepdata/1_exp1/test/LAT_pre_img/'
test_label_path = './3_deepdata/1_exp1/test/LAT_pre_label/'

## npy file ##
npy_path = './3_deepdata/1_exp1/npy/LAT/'
check_model_path = './4_result/exp1/snapshot/LAT/'
predict_path = './4_result/exp1/result/LAT/'

## img ##
img_rows, img_cols = 512, 512
img_type = 'png'

In [4]:
folder = [npy_path, check_model_path, predict_path]
mkfolder(folder)

In [5]:
# create_train_data(data_path, label_path, npy_path, img_rows, img_cols, 'train', img_type)
# create_test_data(test_path, test_label_path, npy_path, img_rows, img_cols, 'test', img_type)

In [6]:
imgs_train, imgs_mask_train, imgs_test = load_data(npy_path+'train.npy', 
                                                   npy_path+'train_label.npy', 
                                                   npy_path+'test.npy')

------------------------------
load train images...
------------------------------
img :  255.0
mask :  255.0
------------------------------
normalization start...
------------------------------
img :  1.0
mask :  1.0
------------------------------
load test images...
------------------------------
(10130, 512, 512, 1)
(10130, 512, 512, 1)
(254, 512, 512, 1)


In [7]:
def get_unet(img_rows, img_cols):
    inputs = Input((img_rows, img_cols,1))
    conv1 = Conv2D(32, (3, 3), activation=None, padding='same')(inputs)
    conv1 = BatchNormalization()(conv1)
    conv1 = Activation('relu')(conv1)
    conv1 = Conv2D(32, (3, 3), activation=None, padding='same')(conv1)
    conv1 = BatchNormalization()(conv1)
    conv1 = Activation('relu')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(64, (3, 3), activation=None, padding='same')(pool1)
    conv2 = BatchNormalization()(conv2)
    conv2 = Activation('relu')(conv2)
    conv2 = Conv2D(64, (3, 3), activation=None, padding='same')(conv2)
    conv2 = BatchNormalization()(conv2)
    conv2 = Activation('relu')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(128, (3, 3), activation=None, padding='same')(pool2)
    conv3 = BatchNormalization()(conv3)
    conv3 = Activation('relu')(conv3)
    conv3 = Conv2D(128, (3, 3), activation=None, padding='same')(conv3)
    conv3 = BatchNormalization()(conv3)
    conv3 = Activation('relu')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(256, (3, 3), activation=None, padding='same')(pool3)
    conv4 = BatchNormalization()(conv4)
    conv4 = Activation('relu')(conv4)
    conv4 = Conv2D(256, (3, 3), activation=None, padding='same')(conv4)
    conv4 = BatchNormalization()(conv4)
    conv4 = Activation('relu')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Conv2D(512, (3, 3), activation=None, padding='same')(pool4)
    conv5 = BatchNormalization()(conv5)
    conv5 = Activation('relu')(conv5)
    conv5 = Conv2D(512, (3, 3), activation=None, padding='same')(conv5)
    conv5 = BatchNormalization()(conv5)
    conv5 = Activation('relu')(conv5)

    up6 = concatenate([Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv5), conv4], axis=3)
    conv6 = Conv2D(256, (3, 3), activation=None, padding='same')(up6)
    conv6 = BatchNormalization()(conv6)
    conv6 = Activation('relu')(conv6)
    conv6 = Conv2D(256, (3, 3), activation=None, padding='same')(conv6)
    conv6 = BatchNormalization()(conv6)
    conv6 = Activation('relu')(conv6)

    up7 = concatenate([Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv6), conv3], axis=3)
    conv7 = Conv2D(128, (3, 3), activation=None, padding='same')(up7)
    conv7 = BatchNormalization()(conv7)
    conv7 = Activation('relu')(conv7)
    conv7 = Conv2D(128, (3, 3), activation=None, padding='same')(conv7)
    conv7 = BatchNormalization()(conv7)
    conv7 = Activation('relu')(conv7)

    up8 = concatenate([Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv7), conv2], axis=3)
    conv8 = Conv2D(64, (3, 3), activation=None, padding='same')(up8)
    conv8 = BatchNormalization()(conv8)
    conv8 = Activation('relu')(conv8)
    conv8 = Conv2D(64, (3, 3), activation=None, padding='same')(conv8)
    conv8 = BatchNormalization()(conv8)
    conv8 = Activation('relu')(conv8)

    up9 = concatenate([Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv8), conv1], axis=3)
    conv9 = Conv2D(32, (3, 3), activation=None, padding='same')(up9)
    conv9 = BatchNormalization()(conv9)
    conv9 = Activation('relu')(conv9)
    conv9 = Conv2D(32, (3, 3), activation=None, padding='same')(conv9)
    conv9 = BatchNormalization()(conv9)
    conv9 = Activation('relu')(conv9)

    conv10 = Conv2D(1, (1, 1), activation='sigmoid')(conv9)

    model = Model(inputs=inputs, outputs=conv10)

    return model

In [8]:
def dice_coef(y_true, y_pred, smooth=1e-6):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def dice_coef_loss(y_true, y_pred):
    return 1-dice_coef(y_true, y_pred)

def sens(y_target, y_pred): # sensitivity, recall
    # clip(t, clip_value_min, clip_value_max) : clip_value_min~clip_value_max 이외 가장자리를 깎아 낸다
    # round : 반올림한다
    y_target_yn = K.round(K.clip(y_target, 0, 1)) # 실제값을 0(Negative) 또는 1(Positive)로 설정한다
    y_pred_yn = K.round(K.clip(y_pred, 0, 1)) # 예측값을 0(Negative) 또는 1(Positive)로 설정한다

    # True Positive는 실제 값과 예측 값이 모두 1(Positive)인 경우이다
    count_true_positive = K.sum(y_target_yn * y_pred_yn) 

    # (True Positive + False Negative) = 실제 값이 1(Positive) 전체
    count_true_positive_false_negative = K.sum(y_target_yn)

    # Recall =  (True Positive) / (True Positive + False Negative)
    # K.epsilon()는 'divide by zero error' 예방차원에서 작은 수를 더한다
    recall = count_true_positive / (count_true_positive_false_negative + K.epsilon())

    # return a single tensor value
    return recall

# multi-GPU

In [9]:
def sch(epoch):
    if epoch>100 and epoch<=250:
        return 0.0001
    elif epoch>250:
        return 0.00001
    else:
        return 0.001

In [10]:
print('-'*30)
print("load unet model")
print('-'*30)

model = get_unet(img_rows, img_cols)
model = multi_gpu_model(model,gpus=2)
model.compile(optimizer=Adam(lr=1e-3), loss=dice_coef_loss, metrics=['accuracy', sens, dice_coef_loss])

------------------------------
load unet model
------------------------------
Instructions for updating:
Colocations handled automatically by placer.


In [11]:
model_checkpoint = ModelCheckpoint(check_model_path+'lat_aug_exp1_{epoch:d}_{loss:f}.h5', monitor='val_dice_coef_loss',verbose=1, save_best_only=True)
# sc = LearningRateScheduler(sch)
reduce_lr = ReduceLROnPlateau(monitor='val_dice_coef_loss', factor=0.8, min_delta = 0.01, patience=5, min_lr=1e-6, verbose=1)
earlystopping = EarlyStopping(monitor='val_dice_coef_loss', patience=10)

In [12]:
print('Fitting model...')
model.fit(imgs_train, imgs_mask_train, batch_size=16, epochs=200, verbose=1, validation_split=0.2, shuffle=True, callbacks=[model_checkpoint, earlystopping])

print('save model')
model.save(predict_path+'lat_aug_exp1.h5')

print('predict test data')
imgs_mask_test = model.predict(imgs_test, batch_size=1, verbose=1)
np.save(predict_path+'prediction_lat_aug_exp1.npy', imgs_mask_test)

Fitting model...
Train on 8104 samples, validate on 2026 samples
Epoch 1/200
8104/8104 [==============================] - 261s 32ms/step - loss: 0.2174 - accuracy: 0.9748 - sens: 0.8947 - dice_coef_loss: 0.2173 - val_loss: 0.1541 - val_accuracy: 0.9840 - val_sens: 0.8669 - val_dice_coef_loss: 0.1539

Epoch 00001: val_dice_coef_loss improved from inf to 0.15392, saving model to ./4_result/exp1/snapshot/LAT/lat_aug_exp1_1_0.217409.h5
Epoch 2/200
8104/8104 [==============================] - 247s 30ms/step - loss: 0.0833 - accuracy: 0.9913 - sens: 0.9293 - dice_coef_loss: 0.0833 - val_loss: 0.1248 - val_accuracy: 0.9865 - val_sens: 0.9018 - val_dice_coef_loss: 0.1247

Epoch 00002: val_dice_coef_loss improved from 0.15392 to 0.12474, saving model to ./4_result/exp1/snapshot/LAT/lat_aug_exp1_2_0.083321.h5
Epoch 3/200
8104/8104 [==============================] - 246s 30ms/step - loss: 0.0651 - accuracy: 0.9931 - sens: 0.9450 - dice_coef_loss: 0.0651 - val_loss: 0.0772 - val_accuracy: 0.9916 -

In [4]:
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import r2_score
import cv2
from keras.models import load_model

name_list=np.load('./3_deepdata/1_exp1/npy/LAT/test_name.npy')
df = pd.DataFrame(columns=['name', 'acc', 'sen', 'spe', 'dsc'])

true_list=np.load('./3_deepdata/1_exp1/npy/LAT/test_label.npy')
true_list=true_list.astype('float32')
true_list = true_list/255.0
true_list[true_list > 0.5] = 1
true_list[true_list <= 0.5] = 0
print(true_list.shape)

pred_list=np.load('./4_result/exp1/result/LAT/prediction_lat_aug_exp1.npy')
pred_list[pred_list > 0.5] = 1
pred_list[pred_list <= 0.5] = 0
# pred_list[pred_list > 127] = 1
# pred_list[pred_list <= 127] = 0

sensitivity=[]
specificity=[]
acc=[]
dsc=[]

for i in range(len(true_list)):
    yt=true_list[i].flatten()
    yp=pred_list[i].flatten()
    mat=confusion_matrix(yt,yp)
    if len(mat) == 2:
        ac=(mat[1,1]+mat[0,0])/(mat[1,0]+mat[1,1]+mat[0,1]+mat[0,0])
        st=mat[1,1]/(mat[1,0]+mat[1,1])
        sp=mat[0,0]/(mat[0,1]+mat[0,0])
        if mat[1,0]+mat[1,1] == 0:
            specificity.append(sp)
            acc.append(ac)
        else:
            sensitivity.append(st)  
            specificity.append(sp)
            acc.append(ac)
    else:
        specificity.append(1)
        acc.append(1)

for i in range(len(true_list)):
    yt=true_list[i]
    yp=pred_list[i]
    if np.sum(yt) != 0 and np.sum(yp) != 0:
        dice = np.sum(yp[yt==1])*2.0 / (np.sum(yt) + np.sum(yp))
        dsc.append(dice)
    df=  df.append({'name':name_list[i], 'acc':ac, 'sen':st, 'spe':sp, 'dsc':dice}, ignore_index=True)

print("complete")     
print("acc avg : {0:0.4f}".format(np.mean(acc)))
print("sensitivity avg : {0:0.4f}".format(np.mean(sensitivity)))
print("specificity avg : {0:0.4f}".format(np.mean(specificity)))
print("dsc avg : {0:0.4f}".format(np.mean(dsc)))

# print("sensitivity min:",np.min(sensitivity))
# print("specificity min:",np.min(specificity))
# print("dsc min:",np.min(dsc))
# print("acc min:",np.min(acc))

# print("sensitivity max:",np.max(sensitivity))
# print("specificity max:",np.max(specificity))
# print("dsc max:",np.max(dsc))
# print("acc max:",np.max(acc))

print('save file')
df.to_csv('./4_result/exp1/result/LAT/lat_aug_exp1.csv')

(254, 512, 512, 1)
complete
acc avg : 0.9975
sensitivity avg : 0.9559
specificity avg : 0.9986
dsc avg : 0.9484
save file


In [3]:
pred_img_path = './4_result/exp1/result/LAT/exp1_img/'
if not os.path.isdir(pred_img_path):
    os.makedirs(pred_img_path)

# pred_list=np.load('./4_ap_pre_pred/ap_pre_exp2.npy')
# pred_list[pred_list > 0.5] = 1
# pred_list[pred_list <= 0.5] = 0

# name_list=np.load('./2_AP_pre_npy/test_pre_name.npy')
print("array to image")
imgs = pred_list
for i in range(imgs.shape[0]):
    img = imgs[i]
    img[img <= 0.5] = 0
    img[img > 0.5] = 255
    img = array_to_img(img)
    img.save(pred_img_path+"%s_pred.png" %(name_list[i]))

array to image


In [4]:
pred_overlay_path = './4_result/exp1/result/LAT/exp1_overlay/'
if not os.path.isdir(pred_overlay_path):
    os.makedirs(pred_overlay_path)
    
print("array to overlay image")
imgs = pred_list
gts = true_list
ori_path = glob.glob('./3_deepdata/1_exp1/test/LAT_pre_img/' + '*.png')

for i in range(imgs.shape[0]):
    img = imgs[i]
    ori_imgs = load_img(ori_path[i])
    ori_imgs = ori_imgs.resize((512,512))
    ori_img = img_to_array(ori_imgs)
    img_name = ori_path[i][ori_path[i].rindex('/')+1:ori_path[i].rindex('.')]
#     print(img_name)
    img[img <= 0.5] = 0
    img[img > 0.5] = 255
    img = img_to_array(img)
    
    ori_img[:,:,0] = img[:,:,0]+ori_img[:,:,0]
    ori_img[:,:,0][ori_img[:,:,0]>255]=255
    ori_img = array_to_img(ori_img)
    ori_img.save(pred_overlay_path+"{}_pred.png".format(img_name))

array to overlay image


In [5]:
pred_overlay_path = './4_result/exp1/result/LAT/exp1_overlay_tpfnfp/'
if not os.path.isdir(pred_overlay_path):
    os.makedirs(pred_overlay_path)
    
print("array to overlay image")
imgs = pred_list
gts = true_list
ori_path = glob.glob('./3_deepdata/1_exp1/test/LAT_pre_img/' + '*.png')

for i in range(imgs.shape[0]):
    img = imgs[i]
    gt = gts[i]
    ori_imgs = load_img(ori_path[i])
    ori_imgs = ori_imgs.resize((512,512))
    ori_img = img_to_array(ori_imgs)
    img_name = ori_path[i][ori_path[i].rindex('/')+1:ori_path[i].rindex('.')]
#     print(img_name)
    img[img <= 0.5] = 0
    img[img > 0.5] = 255
    img = img_to_array(img)
    gt[gt <= 0.5] = 0
    gt[gt > 0.5] = 255
    gt = img_to_array(gt)
    
    ori_img[:,:,0] = img[:,:,0]+ori_img[:,:,0]
    ori_img[:,:,0][ori_img[:,:,0]>255]=255
    
    ori_img[:,:,1] = gt[:,:,0]+ori_img[:,:,1]
    ori_img[:,:,1][ori_img[:,:,1]>255]=255
#     print(np.unique(ori_img[:,:,2]))
    pred_img = array_to_img(ori_img)
    pred_img.save(pred_overlay_path+"{}_pred.png".format(img_name))

array to overlay image


In [6]:
import matplotlib.pyplot as plt
from scipy import ndimage

def max_contour(img):
    contours, _ = cv2.findContours(img, cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)
    tmp =[]
#     print(len(contours))
    for k in range(len(contours)):
        cnt = contours[k]
        mmt = cv2.moments(cnt) 
        tmp.append(float(mmt['m00']))

    max_num = max(tmp)
    index = tmp.index(max_num)
#     contour = contours[index]
    return contours, index
    
def postprocessing(img):
    img = img.astype('uint8')
#     print(img.dtype)
    ret, thresh = cv2.threshold(img,127,255,0)
    img = ndimage.binary_fill_holes(thresh).astype('uint8')
    img = np.expand_dims(img, axis=-1)
#     print(img.shape)
    
    contours, max_cnt_index = max_contour(img)
    
    for i, contour in enumerate(contours):
        if i!=max_cnt_index:
            cv2.fillPoly(img, [contour], color=(0,0,0))
#         else:
#             cv2.fillPoly(img, [contour], color=(255,255,255))
    kernel = np.ones((7,7), np.uint8)
    morph_cnt = 8
#     imshow_plt(img)
#     print(img.shape)
    img_m = img.copy()
    for j in range(morph_cnt):
        img_m = cv2.morphologyEx(img_m, cv2.MORPH_DILATE, kernel)
#     print(img.shape)
#     imshow_plt(img)
    for k in range(morph_cnt):
        img_m = cv2.morphologyEx(img_m, cv2.MORPH_ERODE, kernel)
#     imshow_plt(img)
    return img_m

In [7]:
imgs_path = sorted(glob.glob('./4_result/exp1/result/LAT/exp1_img/*.png'))

post_img_path = './4_result/exp1/result/LAT/exp1_postprocessing/'
if not os.path.isdir(post_img_path):
    os.makedirs(post_img_path)
    
for i, img_path in enumerate(imgs_path):
    img_name = img_path[img_path.rindex('/')+1:img_path.rindex('_')]
#     print(img_name)

#     if i==10:
#         break
    img = cv2.imread(img_path,0)
    post_img = postprocessing(img)
    post_img = cv2.cvtColor(post_img, cv2.COLOR_GRAY2BGR)
#     print(post_img.shape)
#     plt.figure(figsize=(15,15))
#     plt.imshow(post_img[:,:,0], cmap='gray')
#     plt.show()
    
    post_img = array_to_img(post_img)
    post_img.save(post_img_path+"{}_post.png".format(img_name))

In [7]:
pred_overlay_path = './4_result/exp1/result/LAT/exp1_overlay_post/'
if not os.path.isdir(pred_overlay_path):
    os.makedirs(pred_overlay_path)
    
print("array to overlay image")
imgs = pred_list
ori_path = glob.glob('./3_deepdata/1_exp1/test/LAT_pre_img/' + '*.png')

for i in range(imgs.shape[0]):
    img = imgs[i]
    ori_imgs = load_img(ori_path[i])
    ori_imgs = ori_imgs.resize((512,512))
    ori_img = img_to_array(ori_imgs)
    img_name = ori_path[i][ori_path[i].rindex('/')+1:ori_path[i].rindex('.')]
#     print(img_name)
    img[img <= 0.5] = 0
    img[img > 0.5] = 255
    img = img_to_array(img)
    post_img = postprocessing(img)
    post_img = cv2.cvtColor(post_img, cv2.COLOR_GRAY2BGR)
    
    ori_img[:,:,0] = post_img[:,:,0]*255+ori_img[:,:,0]
    ori_img[:,:,0][ori_img[:,:,0]>255]=255
    
    pred_img = array_to_img(ori_img)
    pred_img.save(pred_overlay_path+"{}_pred.png".format(img_name))

array to overlay image


In [8]:
pred_overlay_path = './4_result/exp1/result/LAT/exp1_overlay_post_tpfnfp/'
if not os.path.isdir(pred_overlay_path):
    os.makedirs(pred_overlay_path)
    
print("array to overlay image")
imgs = pred_list
ori_path = glob.glob('./3_deepdata/1_exp1/test/LAT_pre_img/' + '*.png')

for i in range(imgs.shape[0]):
#     if i==1:
#         break
    img = imgs[i]
    ori_imgs = load_img(ori_path[i])
    ori_imgs = ori_imgs.resize((512,512))
    ori_img = img_to_array(ori_imgs)
    img_name = ori_path[i][ori_path[i].rindex('/')+1:ori_path[i].rindex('.')]
#     print(img_name)
    img[img <= 0.5] = 0
    img[img > 0.5] = 255
    img = img_to_array(img)
    post_img = postprocessing(img)
    post_img = cv2.cvtColor(post_img, cv2.COLOR_GRAY2BGR)
    
    ori_img[:,:,0] = post_img[:,:,0]*255+ori_img[:,:,0]
    ori_img[:,:,0][ori_img[:,:,0]>255]=255
    
#     ori_img[:,:,1] = img[:,:,0]+ori_img[:,:,1]
#     ori_img[:,:,1][ori_img[:,:,1]>255]=255
    
    
    ori_img[:,:,1] = img[:,:,0]+ori_img[:,:,1]
    ori_img[:,:,1][ori_img[:,:,1]>255]=255
    
    pred_img = array_to_img(ori_img)
    pred_img.save(pred_overlay_path+"{}_pred.png".format(img_name))

array to overlay image


In [119]:
def imshow_plt(img):
    plt.figure(figsize=(15,15))
    if len(img.shape)==2:
        img = np.expand_dims(img, axis=-1)
    plt.imshow(img[:,:,0], cmap='gray')
    plt.show()

In [33]:
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import r2_score
import cv2
from keras.models import load_model

name_list=np.load('./3_deepdata/1_exp1/npy/LAT/test_name.npy')
df = pd.DataFrame(columns=['name', 'acc', 'sen', 'spe', 'dsc'])

true_list=np.load('./3_deepdata/1_exp1/npy/LAT/test_label.npy')
true_list=true_list.astype('float32')
true_list = true_list/255.0
true_list[true_list > 0.5] = 1
true_list[true_list <= 0.5] = 0
print(true_list.shape)

pred_list=np.array([cv2.imread(path, 0) for path in sorted(glob.glob('./4_result/exp1/result/LAT/exp1_postprocessing/*.png'))])
pred_list = np.expand_dims(pred_list, axis=-1)
pred_list = pred_list/255.0
# pred_list[pred_list > 0.5] = 1
# pred_list[pred_list <= 0.5] = 0
# pred_list[pred_list > 127] = 1
# pred_list[pred_list <= 127] = 0

sensitivity=[]
specificity=[]
acc=[]
dsc=[]

for i in range(len(true_list)):
    yt=true_list[i].flatten()
    yp=pred_list[i].flatten()
    mat=confusion_matrix(yt,yp)
    if len(mat) == 2:
        ac=(mat[1,1]+mat[0,0])/(mat[1,0]+mat[1,1]+mat[0,1]+mat[0,0])
        st=mat[1,1]/(mat[1,0]+mat[1,1])
        sp=mat[0,0]/(mat[0,1]+mat[0,0])
        if mat[1,0]+mat[1,1] == 0:
            specificity.append(sp)
            acc.append(ac)
        else:
            sensitivity.append(st)  
            specificity.append(sp)
            acc.append(ac)
    else:
        specificity.append(1)
        acc.append(1)

# for i in range(len(true_list)):
    yt=true_list[i]
    yp=pred_list[i]
    if np.sum(yt) != 0 and np.sum(yp) != 0:
        dice = np.sum(yp[yt==1])*2.0 / (np.sum(yt) + np.sum(yp))
        dsc.append(dice)
    df=  df.append({'name':name_list[i], 'acc':ac, 'sen':st, 'spe':sp, 'dsc':dice}, ignore_index=True)

print("complete")      
print("acc avg : {0:0.4f}".format(np.mean(acc)))
print("sensitivity avg : {0:0.4f}".format(np.mean(sensitivity)))
print("specificity avg : {0:0.4f}".format(np.mean(specificity)))
print("dsc avg : {0:0.4f}".format(np.mean(dsc)))

# print("sensitivity min:",np.min(sensitivity))
# print("specificity min:",np.min(specificity))
# print("dsc min:",np.min(dsc))
# print("acc min:",np.min(acc))

# print("sensitivity max:",np.max(sensitivity))
# print("specificity max:",np.max(specificity))
# print("dsc max:",np.max(dsc))
# print("acc max:",np.max(acc))

print('save file')
df.to_csv('./4_result/exp1/result/LAT/ap_aug_exp1_post.csv')

(254, 512, 512, 1)
complete
acc avg : 0.9970
sensitivity avg : 0.9660
specificity avg : 0.9978
dsc avg : 0.9393
save file
